# Single-Cell PAGE Tutorial (CRC Demo)

This notebook shows the current single-cell workflow on the bundled CRC AnnData file.


## 1. Imports


In [ ]:
import numpy as np
import anndata
from pypage import GeneSets, SingleCellPAGE


## 2. Load data


In [ ]:
adata = anndata.read_h5ad("../example_data/CRC.h5ad")
adata


In [ ]:
gs = GeneSets.from_gmt("../example_data/c2.all.v2026.1.Hs.symbols.gmt")
print(gs)


## 3. Initialize SingleCellPAGE


In [ ]:
sc = SingleCellPAGE(
    adata=adata,
    genesets=gs,
    function="cmi",
    n_bins=10,
    bin_axis="cell",
    fast_mode=True,
    n_jobs=1,
    filter_redundant=True,
    redundancy_scope="fdr",
    redundancy_fdr=0.05,
)
sc


## 4. Run analysis

For tutorial speed, use a smaller permutation count.


In [ ]:
np.random.seed(42)
results = sc.run(n_permutations=200)
results.head()


## 5. Plot helpers


In [ ]:
ax = sc.plot_consistency_ranking(top_n=20, fdr_threshold=0.05)
ax


In [ ]:
if "X_umap" in sc.embeddings and len(results) > 0:
    top_pathway = results.iloc[0]["pathway"]
    sc.plot_pathway_on_embedding(top_pathway, embedding_key="X_umap")


## 6. CLI equivalent

```bash
pypage-sc --adata ../example_data/CRC.h5ad \
    --gene-column gene \
    --gmt ../example_data/c2.all.v2026.1.Hs.symbols.gmt \
    --groupby PhenoGraph_clusters --fast-mode --n-permutations 1000
```
